In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

# codecademyU - Old
transactions = pd.read_csv('transactions.csv')

#print(transactions.head())
#print(transactions.columns)
print(transactions.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB
None


In [9]:
print(transactions['amount'].describe())

count    6.362620e+06
mean     1.798619e+05
std      6.038582e+05
min      0.000000e+00
25%      1.338957e+04
50%      7.487194e+04
75%      2.087215e+05
max      9.244552e+07
Name: amount, dtype: float64


In [10]:
print(transactions['type'].describe())

count      6362620
unique           5
top       CASH_OUT
freq       2237500
Name: type, dtype: object


In [25]:
# Modify data and put in new column
# Booleans
transactions['isMovement'] = transactions[transactions['type'] == "CASH_OUT"] = 1
transactions['isMovement'] = transactions[transactions['type'] == "TRANSFER"] = 0

transactions['isPayment'] = transactions[transactions['type'] == "PAYMENT"] = 1
transactions['isPayment'] = transactions[transactions['type'] == "DEBIT"] = 0

# Significant difference can trigger fraud alert
transactions['accountDiff'] = transactions['oldbalanceOrg'] - transactions['oldbalanceDest']

print(transactions.info())
#print(transactions['oldbalanceDest'].max())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 14 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
 11  isMovement      int64  
 12  isPayment       int64  
 13  accountDiff     float64
dtypes: float64(6), int64(5), object(3)
memory usage: 679.6+ MB
None
355553416.3


In [31]:
# Select and Split the Data
# Data used to predict
x = transactions[['amount', 'isPayment', 'isMovement', 'accountDiff']]
# Achieving column
y = transactions.isFraud

# Transform X
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x)
X = scaler.transform(x)

# Split data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 27)

In [35]:
# Create and fit the logistic regression model here:
cc_lr = LogisticRegression()
cc_lr.fit(X_train,y_train)

# Print the intercept and coefficients here:
print(cc_lr.coef_)
print(cc_lr.intercept_)

# Save and print the predicted outcomes
y_pred = cc_lr.predict(X_test)
print('Predicted classes: ', y_pred)

# Print out the true outcomes for the test data
print('True classes: ', y_test)

# Print out the predicted probabilities for the test data
print(cc_lr.predict_proba(X_test)[:,1])

[[-96.51395872   0.           0.          -0.5331494 ]]
[-37.14542448]
Predicted classes:  [1 1 1 ... 0 0 1]
True classes:  177267     1
6185488    1
5765101    1
4830191    1
6268533    1
          ..
1712179    1
2633714    1
3739931    0
4484624    0
3260104    1
Name: isFraud, Length: 1908786, dtype: int64
[8.82564167e-01 8.82564167e-01 8.82564167e-01 ... 1.05140726e-06
 2.06080941e-20 8.82564167e-01]


In [41]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

# Print out the confusion matrix
# [True Negatives, False Positives] 
# [False Negatives, True Positives]
print('Confusion Matrix: \n{}'.format(confusion_matrix(y_test, y_pred)))

# Accuracy:
print('Accuracy Score: {:.2f}%'.format(accuracy_score(y_test, y_pred) * 100))

# Precision:
print('Precision Score: {:.2f}%'.format(precision_score(y_test, y_pred) * 100))

# Recall: 
print('Recall Score: {:.2f}%'.format(recall_score(y_test, y_pred) * 100))

# F1 score
print('F1 Score: {:.2f}%'.format(f1_score(y_test, y_pred) * 100))

Confusion Matrix: 
[[ 418305  174136]
 [      0 1316345]]
Accuracy Score: 90.88%
Precision Score: 88.32%
Recall Score: 100.00%
F1 Score: 93.80%


In [45]:
print(x.mean())

amount          37150.070397
isPayment           0.000000
isMovement          0.000000
accountDiff    440404.860656
dtype: float64


In [63]:
# New transaction data
transaction1 = np.array([123456.78, 0.0, 1.0, 54670.1])
transaction2 = np.array([98765.43, 1.0, 0.0, 8524.75])
transaction3 = np.array([543678.31, 1.0, 0.0, 510025.5])

# Create a new transaction
your_transaction = np.array([6472.54, 1.0, 0.0, 55901.23])

# Combine new transactions into a single array
sample_transactions = np.stack((transaction1,transaction2,transaction3,your_transaction))

# Normalize the new transactions
sample_transactions = scaler.transform(sample_transactions)

# Predict fraud on the new transactions
print(cc_lr.predict(sample_transactions))

# Show probabilities on the new transactions
print(cc_lr.predict_proba(sample_transactions))

[0 0 0 0]
[[1.00000000e+000 2.89069094e-056]
 [1.00000000e+000 5.58151289e-045]
 [1.00000000e+000 2.59490108e-248]
 [9.91851073e-001 8.14892685e-003]]


C:\Users\ivanx\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


   step      type    amount     nameOrig  oldbalanceOrg  newbalanceOrig  \
0     1   PAYMENT   9839.64  C1231006815       170136.0       160296.36   
1     1   PAYMENT   1864.28  C1666544295        21249.0        19384.72   
2     1  TRANSFER    181.00  C1305486145          181.0            0.00   
3     1  CASH_OUT    181.00   C840083671          181.0            0.00   
4     1   PAYMENT  11668.14  C2048537720        41554.0        29885.86   

      nameDest  oldbalanceDest  newbalanceDest  isFraud  isFlaggedFraud  
0  M1979787155             0.0             0.0        0               0  
1  M2044282225             0.0             0.0        0               0  
2   C553264065             0.0             0.0        1               0  
3    C38997010         21182.0             0.0        1               0  
4  M1230701703             0.0             0.0        0               0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 co

C:\Users\ivanx\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
